# Reading FOR-instance dataset and creating Partitions

## Show Plots + Outputs

In [ ]:
show = False

## Load data

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

import laspy
import torch
import numpy as np

from src.data import Data, InstanceData
from torch_geometric.nn.pool.consecutive import consecutive_cluster

from src.transforms import *



### label mapping and colors

In [ ]:
FORInstance_NUM_CLASSES = 2

ID2TRAINID = np.asarray([2, 0, 0, 2, 1, 1, 1])

CLASS_NAMES = [
    'Ground and low vegetation',  # 2 Ground, 1 Low vegetation
    'Tree',                       # 4 Stem, 5 Live branches, 6 Woody branches
    'Unknown'                     # 0 Unclassified, 3 Out-points
]

CLASS_COLORS = np.asarray([
    [243, 214, 171],    # Ground and Low vegetation
    [ 70, 115,  66],    # Tree
    [  0,   8, 116]     # Unknown
])

### data loader function

In [ ]:
import time

def read_FORinstance_plot(filepath, xyz=True, intensity=True, 
                           semantic=True, instance=True, remap=True, 
                           max_intensity=None):
    """
    Read a FORinstance plot from a LAS file and return the data object.

    :param filepath: str
        Absolute path to the LAS file
    :param xyz: bool
        Whether XYZ coordinates should be saved in the output Data.pos
    :param intensity: bool
        Whether intensity should be saved in the output Data.rgb
    :param semantic: bool
        Whether semantic labels should be saved in the output Data.y
    :param instance: bool
        Whether instance labels should be saved in the output Data.obj
    :param remap: bool
        Whether semantic labels should be mapped from their FORinstance ID
        to their train ID
    :param max_intensity: float
        Maximum value used to clip intensity signal before normalizing 
        to [0, 1]
    """
    data = Data()
    las = laspy.read(filepath)

    if xyz:
        pos = torch.stack([
            torch.as_tensor(np.array(las[axis]))
            for axis in ["X", "Y", "Z"]], dim=-1)
        pos *= las.header.scale
        pos_offset = pos[0]
        data.pos = (pos - pos_offset).float()
        data.pos_offset = pos_offset
    
    intensity_remaped = True
    if intensity:
        data.intensity = torch.FloatTensor(
            las['intensity'].astype('float32')
        )
        if intensity_remaped:
            if max_intensity is None:
                max_intensity = data.intensity.max()
            data.intensity = data.intensity.clip(min=0, max=max_intensity) / max_intensity

    if semantic:
        y = torch.LongTensor(np.array(las['classification']))
        data.y = torch.from_numpy(ID2TRAINID)[y] if remap else y

    if instance:
        idx = torch.arange(data.num_points)
        obj = torch.LongTensor(np.array(las['treeID']))
        
        y = torch.LongTensor(np.array(las['classification']))
        y = torch.from_numpy(ID2TRAINID)[y] if remap else y

        if remap:
            ground_mask = (obj == 0) & (y == 0)
            low_veg_mask = (obj == 0) & (y == 1)
            if low_veg_mask.any() or ground_mask.any():
                ground_instance_label = obj.max().item() + 1
                low_veg_instance_label = ground_instance_label  # for separate ground and low vegetation classes: ground_instance_label + 1
                obj[ground_mask] = ground_instance_label
                obj[low_veg_mask] = low_veg_instance_label

        obj = consecutive_cluster(obj)[0]
        count = torch.ones_like(obj)

        data.obj = InstanceData(idx, obj, count, y, dense=True)
    
    return data

In [ ]:
RMIT_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/forinstance/raw/RMIT/train.las"
RMIT_data = read_FORinstance_plot(RMIT_filepath, instance=True)

In [ ]:
RMIT_data.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS)

In [ ]:
CULS_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/CULS/plot_3_annotated.las"
CULS_data = read_FORinstance_plot(CULS_filepath, instance=True)
NIBIO_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/NIBIO/plot_10_annotated.las"
NIBIO_data = read_FORinstance_plot(NIBIO_filepath, instance=True)
RMIT_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/RMIT/train.las"
RMIT_data = read_FORinstance_plot(RMIT_filepath, instance=True)
SCION_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/SCION/plot_35_annotated.las"
SCION_data = read_FORinstance_plot(SCION_filepath, instance=True)
TUWIEN_filepath = "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/TUWIEN/train.las"
TUWIEN_data = read_FORinstance_plot(TUWIEN_filepath, instance=True)

In [ ]:
if show:
    CULS_data.show(keys=['intensity'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)
    NIBIO_data.show(keys=['intensity'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)
    RMIT_data.show(keys=['intensity'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)
    SCION_data.show(keys=['intensity'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)
    TUWIEN_data.show(keys=['intensity'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)

### Checking intensity and x, y, z values

In [ ]:
if False:
    print(data.intensity)
    import matplotlib.pyplot as plt

    # plot title
    plt.title('Intensity distribution')
    plt.hist(data.intensity.numpy(), bins=10)
    plt.show()
    print(data.intensity.min())

    # plot title
    plt. title('non remaped intensity distribution')
    las = laspy.read(filepath)
    intensity = torch.FloatTensor(
                las['intensity'].astype('float32'))
    plt.hist(intensity.numpy(), bins=10)
    plt.show()

    print(intensity.min())

    # x,y, z values
    print(data.pos[:, 2].min())
    print(data.pos[:, 2].max())



## Tiling
The FOR-instance dataset has already quite small tiles in each file, thus additional tiling of the point cloud in every file is not necessary.

## Partition generation

### Voxelization
`voxel`: size of the voxels in the partitions

In [ ]:
def calc_point_density(voxel_size=0.1, data=None):
    """Calculate the point density for each point in the data.

    :param voxel_size: float
        The size of the voxel used to calculate the point density
    :param data: Data
        The data object containing the point cloud
    :return: vol_density
        points per m^3
    :return: point_ratio
        estimated ratio of points in the voxelized data to the original data
    """
    
    data_voxelized = GridSampling3D(size=voxel_size)(data)
    voxel_ratio = data.num_nodes / data_voxelized.num_nodes
    data_voxelized_1m = GridSampling3D(size=1)(data)
    vol_density = data_voxelized.num_nodes / data_voxelized_1m.num_nodes

    return vol_density, voxel_ratio

def point_density_experiments(voxel_size = 0.1):
    """Calculate the point density for each point cloud in the dataset.
    :param voxel_size: float
        The size of the voxel used to calculate the point density
    """
    CULS_density, CULS_ratio = calc_point_density(voxel_size, CULS_data)
    NIBIO_density, NIBIO_ratio = calc_point_density(voxel_size, NIBIO_data) 
    RMIT_density, RMIT_ratio = calc_point_density(voxel_size, RMIT_data)
    SCION_density, SCION_ratio = calc_point_density(voxel_size, SCION_data)
    TUWIEN_density, TUWIEN_ratio = calc_point_density(voxel_size, TUWIEN_data)

    print(f"---------------------- ")
    print(f"voxel_size = {voxel_size}")
    print(f"CULS: density = {CULS_density:.2f} points/m^3, voxel ratio = {CULS_ratio:.2f}")
    print(f"NIBIO: density = {NIBIO_density:.2f} points/m^3, voxel ratio = {NIBIO_ratio:.2f}")
    print(f"RMIT: density = {RMIT_density:.2f} points/m^3, voxel ratio = {RMIT_ratio:.2f}")
    print(f"SCION: density = {SCION_density:.2f} points/m^3, voxel ratio = {SCION_ratio:.2f}")
    print(f"TUWIEN: density = {TUWIEN_density:.2f} points/m^3, voxel ratio = {TUWIEN_ratio:.2f}")

In [ ]:
if show:
    point_density_experiments(voxel_size = 0.05)
    point_density_experiments(voxel_size = 0.1)
    point_density_experiments(voxel_size = 0.2)
    point_density_experiments(voxel_size = 0.5)
    point_density_experiments(voxel_size = 1)


In [ ]:
def voxelize_all_data(voxel_size):
    """Voxelize all the data in the dataset using the given voxel size.
    :param voxel_size: float
        The size of the voxel used to calculate the point density
    """
    CULS_voxelized = GridSampling3D(size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)(CULS_data)
    NIBIO_voxelized = GridSampling3D(size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)(NIBIO_data)
    RMIT_voxelized = GridSampling3D(size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)(RMIT_data)
    SCION_voxelized = GridSampling3D(size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)(SCION_data)
    TUWIEN_voxelized = GridSampling3D(size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)(TUWIEN_data)

    return CULS_voxelized, NIBIO_voxelized, RMIT_voxelized, SCION_voxelized, TUWIEN_voxelized

In [ ]:
voxel_size = 0.5 # try with 0.05, 0.1, 0.2
CULS_voxelized, NIBIO_voxelized, RMIT_voxelized, SCION_voxelized, TUWIEN_voxelized = voxelize_all_data(voxel_size)

In [ ]:
if show:
    RMIT_voxelized.show(max_points=RMIT_voxelized.num_points, class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)

### Neighbor search
`k`: num nearest neighbors  
`r_max`: search nearest neighbors within this radius


Searches for `k` nearest neighbors of each point, within a maximum radius of `r_max`. Contrary to basic K-NN search, the radius constraint prevents spurious neighborhoods for very sparse areas of the point cloud. By design, this approach implies **points may not all have the same number of neighbors**, depending on the local geometry and density.

The neigbors are used for two things in the preprocessing pipeline:
- computing local geometric features with `PointFeatures`, later used by `CutPursuitPartition` as pointwise signal for the superpoint partition
- computing the adjacency graph with `AdjacencyGraph`, later used by `CutPursuitPartition` as the graph on which the superpoint partition is computed

In [ ]:
def knn_all_data(k=25, r_max=2):
    """Find the k-nearest neighbors for all points in the dataset.
    :param k: int
        The number of neighbors to find
    :param r_max: float
        The maximum distance to consider when searching for neighbors
    """
    CULS_knn = KNN(k, r_max)(CULS_voxelized)
    NIBIO_knn = KNN(k, r_max)(NIBIO_voxelized)
    RMIT_knn = KNN(k, r_max)(RMIT_voxelized)
    SCION_knn = KNN(k, r_max)(SCION_voxelized)
    TUWIEN_knn = KNN(k, r_max)(TUWIEN_voxelized)

    return CULS_knn, NIBIO_knn, RMIT_knn, SCION_knn, TUWIEN_knn

In [ ]:
k = 25
r_max = 2
CULS_knn, NIBIO_knn, RMIT_knn, SCION_knn, TUWIEN_knn = knn_all_data(k, r_max)

### Elevation Estimation

`threshold`: ground as a planar surface located within `threshold` of the lowest point in the cloud.  
`scale`: Pointwise distance to the plane is normalized by `scale`

`GroundElevation` is used to look for the ground among the points, to then infer point `elevation`. Indeed, the elevation is a more informative feature than the `z` coordinate of points for semantic parsing. For real-life large point cloud acquisitions, the absolute `z` value usually carries no meaning, but the _relative `z`_ with respect to the ground does (the same holds for absolute `x` and `y` values).

In [ ]:
def elevation_all_data(threshold=5, scale=20):
    """Calculate the elevation features for all points in the dataset.
    :param threshold: float
        The threshold used to calculate the elevation features
    :param scale: float
        The scale used to calculate the elevation features
    """
    CULS_elevation = GroundElevation(threshold, scale)(CULS_knn)
    NIBIO_elevation = GroundElevation(threshold, scale)(NIBIO_knn)
    RMIT_elevation = GroundElevation(threshold, scale)(RMIT_knn)
    SCION_elevation = GroundElevation(threshold, scale)(SCION_knn)
    TUWIEN_elevation = GroundElevation(threshold, scale)(TUWIEN_knn)

    return CULS_elevation, NIBIO_elevation, RMIT_elevation, SCION_elevation, TUWIEN_elevation
    

In [ ]:
threshold = 5
scale = 20
CULS_elevation, NIBIO_elevation, RMIT_elevation, SCION_elevation, TUWIEN_elevation = elevation_all_data(threshold, scale)

In [ ]:
if True:
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Plot for CULS elevation
    g = sns.displot(CULS_elevation.elevation)
    g.figure.suptitle("CULS Elevation Distribution")
    plt.show()

    # Plot for NIBIO elevation
    g = sns.displot(NIBIO_elevation.elevation)
    g.figure.suptitle("NIBIO Elevation Distribution")
    plt.show()

    # Plot for RMIT elevation
    g = sns.displot(RMIT_elevation.elevation)
    g.figure.suptitle("RMIT Elevation Distribution")
    plt.show()

    # Plot for SCION elevation
    g = sns.displot(SCION_elevation.elevation)
    g.figure.suptitle("SCION Elevation Distribution")
    plt.show()

    # Plot for TUWIEN elevation
    g = sns.displot(TUWIEN_elevation.elevation)
    g.figure.suptitle("TUWIEN Elevation Distribution")
    plt.show()

    CULS_elevation.show(keys=['elevation'], class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)


### Pointwise local geometric features
`PointFeatures` computes some handcrafted geometric features characterizing each point's neighborhood. The following features are currently supported:
- density
- linearity
- planarity
- scattering
- verticality
- normal
- length
- surface
- volume
- curvature
- (RGB color)  
- (HSV color)  
- (LAB color)  

These features should be computed with the superpoint partition in mind: these will be the **criteria based on which points will or will not grouped together** by the cut-pursuit algorithm.

Note that the robustness and expressivity of these computed geometric features will depend on your `KNN` parametrization.

`PointFeatures` supports various strategies for geometric computation. By default, all neighbors produced by `KNN` will be used.

>One may also specify `PointFeatures(k_min=...)` below which a point will receive `0` geometric features, to mitigate the low-quality features for too-small neighborhoods. Besides, `PointFeatures(k_step=..., k_min_search=...)` will search for the optimal neighborhood size among available neighbors for each point, based on eigenfeatures entropy

In [ ]:
def pointfeatures_all_data(features):
    """
    :params features: list of str
    """
    CULS_features = PointFeatures(keys=features)(CULS_elevation)
    NIBIO_features = PointFeatures(keys=features)(NIBIO_elevation)
    RMIT_features = PointFeatures(keys=features)(RMIT_elevation)
    SCION_features = PointFeatures(keys=features)(SCION_elevation)
    TUWIEN_features = PointFeatures(keys=features)(TUWIEN_elevation)

    return CULS_features, NIBIO_features, RMIT_features, SCION_features, TUWIEN_features

In [ ]:
features = ('density', 'linearity', 'planarity', 'scattering', 'verticality', )
CULS_features, NIBIO_features, RMIT_features, SCION_features, TUWIEN_features = pointfeatures_all_data(features)

In [ ]:
if True:
    CULS_features.show(keys=CULS_features.keys, class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)

In [ ]:
g = sns.displot(CULS_features.density)
g.figure.suptitle("CULS Denstiy Distribution")
plt.show()

In [ ]:
dmax = CULS_features.neighbor_distance.max(dim=1).values
k = CULS_features.neighbor_index.ge(0).sum(dim=1)
CULS_features.density = (k / 4 ** 2).view(-1, 1)

In [ ]:
sns.displot(dmax)

In [ ]:
sns.displot(k)

In [ ]:
sns.displot(density)

### Adjacency graph
`k`: use edges of the `k` nearest neighbors  
`w`: edge weights

computes the adjacency graph based on which the superpoint partition will be computed. It is relying on the output of `KNN` to find neighbors for each point. `AdjacencyGraph(k=..., w=...)` will store edges for the `k`-NN graph in `Data.edge_index`, along with edge weights in `Data.edge_attr` to be used in the partition (the larger an edge's weight the harder to separate the corresponding points).

In [ ]:
def adjacency_graph_all_data(k=10, w=1):
    """Create the adjacency graph for all points in the dataset.
    :param k: int
        The number of neighbors to consider when creating the graph
    :param w: float
        The weight to assign to the edges
    """
    CULS_graph = AdjacencyGraph(k, w)(CULS_features)
    NIBIO_graph = AdjacencyGraph(k, w)(NIBIO_features)
    RMIT_graph = AdjacencyGraph(k, w)(RMIT_features)
    SCION_graph = AdjacencyGraph(k, w)(SCION_features)
    TUWIEN_graph = AdjacencyGraph(k, w)(TUWIEN_features)

    return CULS_graph, NIBIO_graph, RMIT_graph, SCION_graph, TUWIEN_graph

In [ ]:
k = 10
w = 1
CULS_graph, NIBIO_graph, RMIT_graph, SCION_graph, TUWIEN_graph = adjacency_graph_all_data(k, w)

### Add keys to x
Before computing the partition, we need to move to the `x` attribute all the features that we want to use for the partition (`CutPursuitPartition` will blindly use whatever it finds `x`). To this end, we will use the `AddKeysTo` transform.

You can play with the features used with `AddKeysTo` and `CutPursuitPartition` parameters, and see how it impacts your partition metrics.

In [ ]:
def add_keys_all_data(features):
    """Add the keys to the data object for all points in the dataset."""
    CULS_graph_x = AddKeysTo(keys=features, to='x', delete_after=False)(CULS_graph)
    NIBIO_graph_x = AddKeysTo(keys=features, to='x', delete_after=False)(NIBIO_graph)
    RMIT_graph_x = AddKeysTo(keys=features, to='x', delete_after=False)(RMIT_graph)
    SCION_graph_x = AddKeysTo(keys=features, to='x', delete_after=False)(SCION_graph)
    TUWIEN_graph_x  = AddKeysTo(keys=features, to='x', delete_after=False)(TUWIEN_graph)

    return CULS_graph_x, NIBIO_graph_x, RMIT_graph_x, SCION_graph_x, TUWIEN_graph_x

In [ ]:
features_to_x = ('elevation', 'linearity', 'planarity', 'scattering', 'verticality')
CULS_graph_x, NIBIO_graph_x, RMIT_graph_x, SCION_graph_x, TUWIEN_graph_x = add_keys_all_data(features_to_x)

### Hierarchical partition
`regularization`: List of increasing float values determining the granularity of hierarchical superpoint partitions.  
`spatial_weight`: Float value indicating the importance of point coordinates relative to point features in grouping points.  
`k_adjacency`: Integer preventing superpoints from being isolated.  
`cutoff`: Integer specifying the minimum number of points in each superpoint, ensuring small superpoints are merged with others.  


`CutPursuitPartition` is where the actual superpoint partition occurs. A regularization term rules the trade-off between "many-superpoint-with-homogeneous-content" and "few-superpoints-with-heterogenous-content".

In `CutPursuitPartition(regularization=..., spatial_weight=..., k_adjacency=..., cutoff=...)`, `regularization` carries a list of increasing float values for coarser and coarser hierarchical superpoint partition levels. `spatial_weight` indicates how much importance the point coordinates play with respect point features, when grouping points: the larger the weight, the more spatial coordinates take over, the more tesselated-looking the partition. `k_adjacency` prevents superpoints from staying isolated. `cutoff` rules the minimum number of points in each superpoint partition level: too-small superpoint will be merged with other superpoints.

In [ ]:
def create_nag_all_data(regularization, spatial_weight, cutoff, iterations, k_adjacency):
    CULS_nag = CutPursuitPartition(regularization=regularization, spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)(CULS_graph_x)
    NIBIO_nag = CutPursuitPartition(regularization=regularization, spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)(NIBIO_graph_x)
    RMIT_nag = CutPursuitPartition(regularization=regularization, spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)(RMIT_graph_x)
    SCION_nag = CutPursuitPartition(regularization=regularization, spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)(SCION_graph_x)
    TUWIEN_nag = CutPursuitPartition(regularization=regularization, spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)(TUWIEN_graph_x)
    
    return CULS_nag, NIBIO_nag, RMIT_nag, SCION_nag, TUWIEN_nag

In [ ]:
regularization=[0.1, 0.2]
spatial_weight=[0.1, 0.01]
cutoff=[10, 30]
iterations=15
k_adjacency=10
CULS_nag, NIBIO_nag, RMIT_nag, SCION_nag, TUWIEN_nag = create_nag_all_data(regularization, spatial_weight, cutoff, iterations, k_adjacency)

In [ ]:
def output_level_ratios():
    print("LEVEL RATIOS")
    print("CULS: ", CULS_nag.level_ratios)
    print("NIBIO: ", NIBIO_nag.level_ratios)
    print("RMIT: ", RMIT_nag.level_ratios)
    print("SCION: ", SCION_nag.level_ratios)
    print("TUWIEN: ", TUWIEN_nag.level_ratios)

In [ ]:
output_level_ratios()

In [ ]:
def oracle_performance_all_data():
    print("CULS:", CULS_nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes))
    print("NIBIO:", NIBIO_nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes))
    print("RMIT:", RMIT_nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes))
    print("SCION:", SCION_nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes))
    print("TUWIEN:", TUWIEN_nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes))

    print("CULS:", CULS_nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))
    print("NIBIO:", NIBIO_nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))
    print("RMIT:", RMIT_nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))
    print("SCION:", SCION_nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))
    print("TUWIEN:", TUWIEN_nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))

    print("CULS:", CULS_nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))
    print("NIBIO:", NIBIO_nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))
    print("RMIT:", RMIT_nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))
    print("SCION:", SCION_nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))
    print("TUWIEN:", TUWIEN_nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))

In [ ]:
oracle_performance_all_data()

In [ ]:
CULS_nag.show(class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)